## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [17]:
# Today's cameras all use lenses which make 2D image distort, so called camera calibration problem
# There are two common distortion:
# (1) Radial distortion, which makes stright lines appear curved, or objects appear more or less curved 
#     than the they actually are.
# (2) tangential distorion, which occurs 
#     because image taking lense is not aligned perfectly parrllel to the imaging plane.
#
# So this part is the practice to undistort the images effected by both radial and tangential distortion.

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt5

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,9,0)
# use numpy mgrid function to generate the coordinates values for a given grid size.
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners in distorted calibration images.
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard's inner corners,
    # (x, y) should pass only points where two black and two white squares intersects.
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners to see what was detected.
#         img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#         cv2.imshow('img',img)
#         cv2.waitKey(500)
        

# cv2.destroyAllWindows()


import pickle

img = cv2.imread('../camera_cal/calibration1.jpg')

# Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
#
# feed objpoints, imgpoints, and shape of the grayscale image into calibrateCamera function 
# to calculate the distortion cofficients (dist)
# and the camera matrix that we need to transform 3D object points to 2D image points
#
# mtx: Camera Matrix, which only depends on the camera only. so once calculated, it can be stored for future purposes.
# dist: Distortion coefficients, which depends on samples. so once calculated, it can be reused onto the same things 
#       as samples with different angles and orientation.
def store_dist_and_mtx(img, objpoints, imgpoints):
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    print("mtx: ", mtx)
    print("dist: ", dist)
    # we only calculate once and store it into pickle 
    # so that we could reuse mtx and dist on every chessboard images in camera_cal folder.
    dist_pickle = {}
    dist_pickle['mtx'] = mtx
    dist_pickle['dist'] = dist
    pickle.dump( dist_pickle, open('mtx_and_dist_pickle.p', 'wb'))

store_dist_and_mtx(img, objpoints, imgpoints)

## And so on and so forth...

In [22]:
# Apply a distortion correction to raw images.
with open('mtx_and_dist_pickle.p', mode='rb') as f:
    dist_pickle = pickle.load(f)
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

%matplotlib inline

for img in images:
    index = img.rfind('/') + 1
    filename = img[index:]
    img = cv2.imread(img)
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite('../output_images/undistort_images/'+filename, undist_img)

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
# ax1.imshow(img)
# ax1.set_title('Original Image', fontsize=50)
# ax2.imshow(undistorted)
# ax2.set_title('Undistorted Image', fontsize=50)
# plt.subplots_adjust(left=0, right=1, top=0.9, bottom=0.)